In [193]:
import warnings

import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl import load_workbook

warnings.filterwarnings('ignore')

# Find closest Train Station and Tram Stop to Tracts' centroids

The cirty of Melbourne is dived in tracts formally known as Statistical Areas Level 1 (SA1), each of said tracts is a polygon. We seek to asociate all tracts with the topolofical characteristics of the stations closest to them. The closest tram stops and the closest train station. The influence of any other stop/station that is not that of the easiest access is ignored. This access is defines by the distance between a station and the centroid of a tract.

In [194]:
crs = 7899
tracts_df = gpd.read_file('Data/Shapefiles/Great_Melbourne/SA1_2016_GM/SA1_2016_GM.shp').to_crs(crs)
tram_df = gpd.read_file('Data/Shapefiles/Public_Transport/tram_open_vic/PTV/PTV_METRO_TRAM_STOP.shp') \
          .to_crs(crs)

# Calculate the centroid of each tract-polygon
tracts_df['centroids'] = tracts_df['geometry'].centroid

In [195]:
# Function to find the closest stop to the centroid of a tract

def closest_stop (polygons, stops, col_stop_ID, name = ''):
    
    name1 = 'closest_stop_' + name
    name2 = 'dist_closest_stop_' + name
    closest_stops = []
    closest_dist = []
    for unit in polygons['centroids']:
        
        distances = stops['geometry'].distance(unit)
        shortest = min(distances)
        position = distances.index[distances == shortest].tolist()
        closest_stops.append(stops.loc[position,col_stop_ID].values[0])
        closest_dist.append(shortest)
        
    polygons[name1] = closest_stops
    polygons[name2] = closest_dist
    
    return polygons

## 2011 and 2016

In [196]:
tracts_stops_df = closest_stop(tracts_df, 
                               tram_df, 
                               col_stop_ID ='STOP_ID',
                               name='Tram')

train_df = gpd.read_file('Data/Shapefiles/Public_Transport/train_aurin/shp/' 
                         + '4e0dbd07-7f8b-4da8-9c7f-1a84d2ab27fc.shp').to_crs(crs)

tracts_tram_train = closest_stop(tracts_stops_df, 
                                 train_df, 
                                 col_stop_ID ='STOP_ID', 
                                 name='Train')

In [112]:
#tracts_tram_train.to_csv('tracts_closest_tram_train_stops_11_16.csv')

## 2006

In [197]:
# Tram stations are considered to remain unchanged in all three census years
tracts_stops_df = closest_stop(tracts_df, 
                               tram_df, 
                               col_stop_ID ='STOP_ID', 
                               name='Tram')

# Use mask of existing train stations in 2006
train_df = gpd.read_file('Data/Shapefiles/Public_Transport/train_aurin/shp/' +
                         '4e0dbd07-7f8b-4da8-9c7f-1a84d2ab27fc.shp').to_crs(crs)

mask_train_stops_2006 = pd.read_csv('Data/PT_Network_Features/train_stops_2006.csv')

train_df_trimmed = merge_str(train_df, 
                             mask_train_stops_2006[['STOP_ID']], 
                             'STOP_ID', 
                             'STOP_ID', 
                             how='right')

tracts_tram_train_2006 = closest_stop(tracts_stops_df, 
                                      train_df_trimmed, 
                                      col_stop_ID ='STOP_ID', 
                                      name='Train')

In [191]:
#tracts_tram_train_2006.to_csv('tracts_closest_tram_train_stops_2006.csv')

# Network construction and properties

In [165]:
# Function to obtain a dataframe with the centrality measures given a set of edges in a file

def properties_from_edges(file_path, sheet=0, header_row=0, cols=[0,1]):
    edges_df = read_table(file_path, sheet=1)
    
    PT_net = nx.Graph()
    
    # load edges of the undirected graph
    for index, row in edges_df.iterrows():
        PT_net.add_edge(row[cols[0]],row[cols[1]])
    
    # Get centrality measures
    dict_bet = nx.betweenness_centrality(PT_net)
    dict_clos = nx.closeness_centrality(PT_net)
    dict_deg = nx.degree_centrality(PT_net)
    
    # Create dataframe preserving the order in the graph's nodes
    rows = []
    for station in PT_net.nodes:
        rows.append([station, dict_deg[station], dict_bet[station], dict_clos[station]])
        
    outcome = pd.DataFrame(rows, columns =['Stations', 'Degree', 'Betweenness', 'Closeness'])
    
    return outcome

## Tram

In [198]:
file_path = 'Data/PT_Network_Features/tram_edges.xlsx'

tram_properties = properties_from_edges(file_path, 
                                        sheet=1)

In [ ]:
#tram_properties.to_csv('Data/PT_Network_Features/tram_properties.csv')

## Train

In [199]:
file_path = 'Data/PT_Network_Features/train_edges_2006.xlsx'

train_properties = properties_from_edges(file_path, 
                                         sheet=1, 
                                         cols=[0,2])

In [ ]:
#train_properties.to_csv('Data/PT_Network_Features/train_properties_2006.csv')